<a href="https://colab.research.google.com/github/Aasthapriy44/Java/blob/main/1BM22Cs003_DFS_IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import copy
from heapq import heappush, heappop


row = [1, 0, -1, 0]
col = [0, -1, 0, 1]

class Node:
    def __init__(self, parent, mat, empty_tile_pos, cost, level):
        self.parent = parent  # Parent node
        self.mat = mat  # Current state of the matrix
        self.empty_tile_pos = empty_tile_pos  # Position of the empty tile
        self.cost = cost  # Number of misplaced tiles
        self.level = level  # Depth of the node in the search tree

    def __lt__(self, other):
        return self.cost < other.cost  # For priority queue sorting


def calculate_cost(mat, final):
    count = 0
    for i in range(3):
        for j in range(3):
            if (mat[i][j] != 0 and mat[i][j] != final[i][j]):
                count += 1
    return count

# Create a new node based on movement
def new_node(mat, empty_tile_pos, new_empty_tile_pos, level, parent, final):
    new_mat = copy.deepcopy(mat)
    x1, y1 = empty_tile_pos
    x2, y2 = new_empty_tile_pos
    new_mat[x1][y1], new_mat[x2][y2] = new_mat[x2][y2], new_mat[x1][y1]  # Swap the tiles
    cost = calculate_cost(new_mat, final)
    return Node(parent, new_mat, new_empty_tile_pos, cost, level)

# Print the puzzle matrix
def print_matrix(mat):
    for row in mat:
        print(" ".join(str(x) for x in row))
    print()

# Check if a move is within the grid
def is_safe(x, y):
    return 0 <= x < 3 and 0 <= y < 3  # Fixed for 3x3 grid

# Print the path from the initial state to the solution
def print_path(root):
    if root is None:
        return
    print_path(root.parent)
    print_matrix(root.mat)

# Solve the 8-puzzle using Branch and Bound
def solve(initial, empty_tile_pos, final):
    pq = []  # Priority Queue
    cost = calculate_cost(initial, final)
    root = Node(None, initial, empty_tile_pos, cost, 0)
    heappush(pq, root)  # Add root to the queue

    while pq:
        # Get the node with the least cost
        minimum = heappop(pq)

        # If we reached the goal state
        if minimum.cost == 0:
            print("Solution path:")
            print_path(minimum)
            return

        # Generate all possible children
        for i in range(4):
            new_tile_pos = [minimum.empty_tile_pos[0] + row[i],
                            minimum.empty_tile_pos[1] + col[i]]
            if is_safe(new_tile_pos[0], new_tile_pos[1]):
                child = new_node(minimum.mat, minimum.empty_tile_pos, new_tile_pos,
                                 minimum.level + 1, minimum, final)
                heappush(pq, child)  # Add child to the priority queue

initial = [[1, 2, 3],
           [5, 6, 0],
           [7, 8, 4]]

final = [[1, 2, 3],
         [5, 8, 6],
         [0, 7, 4]]

empty_tile_pos = [1, 2]


solve(initial, empty_tile_pos, final)


Solution path:
1 2 3
5 6 0
7 8 4

1 2 3
5 0 6
7 8 4

1 2 3
5 8 6
7 0 4

1 2 3
5 8 6
0 7 4



In [12]:
class PuzzleState:
    def __init__(self, board, zero_pos, moves):
        self.board = board
        self.zero_pos = zero_pos
        self.moves = moves

    def __hash__(self):
        return hash(tuple(tuple(row) for row in self.board))

    def __eq__(self, other):
        return self.board == other.board

def is_goal_state(board):
    goal = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    return board == goal

def get_possible_moves(zero_pos):
    moves = []
    row, col = zero_pos
    if row > 0: moves.append((-1, 0))
    if row < 2: moves.append((1, 0))
    if col > 0: moves.append((0, -1))
    if col < 2: moves.append((0, 1))
    return moves

def make_move(board, zero_pos, move):
    new_board = [row[:] for row in board]
    new_zero_pos = (zero_pos[0] + move[0], zero_pos[1] + move[1])

    new_board[zero_pos[0]][zero_pos[1]], new_board[new_zero_pos[0]][new_zero_pos[1]] = \
        new_board[new_zero_pos[0]][new_zero_pos[1]], new_board[zero_pos[0]][zero_pos[1]]
    return new_board, new_zero_pos

def dfs(initial_state):
    stack = [initial_state]
    visited = set()

    while stack:
        current_state = stack.pop()
        visited.add(current_state)

        if is_goal_state(current_state.board):
            return current_state.moves
        possible_moves = get_possible_moves(current_state.zero_pos)

        for move in possible_moves:
            new_board, new_zero_pos = make_move(current_state.board, current_state.zero_pos, move)
            new_state = PuzzleState(new_board, new_zero_pos, current_state.moves + [move])

            if new_state not in visited:
                stack.append(new_state)

    return None


initial_board = [
    [0,1, 2],
    [3,4, 5],
    [6, 7, 8]  # Initial state
]
initial_zero_pos = (0, 0)
initial_state = PuzzleState(initial_board, initial_zero_pos, [])

solution_moves = dfs(initial_state)

if solution_moves is not None:
    print("Solution found with moves:", solution_moves)
else:
    print("No solution found.")

Solution found with moves: [(0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (0, -1), (1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, -